In [3]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
!ls -l ../data

total 23600
-rw-r--r-- 1 user 197121 11663225 May 17 23:21 fhv_tripdata_2022-01.parquet
-rw-r--r-- 1 user 197121 12497217 May 17 23:22 fhv_tripdata_2022-02.parquet


In [16]:
df = pd.read_parquet('../data/fhv_tripdata_2022-01.parquet')
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2022-01-01 00:31:00,2022-01-01 01:05:00,NaN,NaN,None,B00009
1,B00009,2022-01-01 00:37:00,2022-01-01 01:05:00,NaN,NaN,None,B00009
2,B00037,2022-01-01 00:56:37,2022-01-01 01:06:11,NaN,85.0,None,B00037
3,B00037,2022-01-01 00:19:54,2022-01-01 00:30:47,NaN,85.0,None,B00037
4,B00037,2022-01-01 00:41:49,2022-01-01 00:52:16,NaN,188.0,None,B00037


In [17]:
print("Num of rows:", len(df))

Num of rows: 1143691


In [18]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [24]:
df.duration.mean()

22.550803232691358

In [27]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
len(df)

1084708

In [28]:
print("Dropped rows count:", (1143691 - 1084708))

Dropped rows count: 58983


In [29]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2022-01-01 00:31:00,2022-01-01 01:05:00,NaN,NaN,None,B00009,34.000000
1,B00009,2022-01-01 00:37:00,2022-01-01 01:05:00,NaN,NaN,None,B00009,28.000000
2,B00037,2022-01-01 00:56:37,2022-01-01 01:06:11,NaN,85.0,None,B00037,9.566667
3,B00037,2022-01-01 00:19:54,2022-01-01 00:30:47,NaN,85.0,None,B00037,10.883333
4,B00037,2022-01-01 00:41:49,2022-01-01 00:52:16,NaN,188.0,None,B00037,10.450000


In [30]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)

In [31]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2022-01-01 00:31:00,2022-01-01 01:05:00,-1.0,-1.0,None,B00009,34.000000
1,B00009,2022-01-01 00:37:00,2022-01-01 01:05:00,-1.0,-1.0,None,B00009,28.000000
2,B00037,2022-01-01 00:56:37,2022-01-01 01:06:11,-1.0,85.0,None,B00037,9.566667
3,B00037,2022-01-01 00:19:54,2022-01-01 00:30:47,-1.0,85.0,None,B00037,10.883333
4,B00037,2022-01-01 00:41:49,2022-01-01 00:52:16,-1.0,188.0,None,B00037,10.450000


In [39]:
print('Fraction of missing values in the PUlocationID coumn:',\
      df['PUlocationID'].value_counts()[-1] / len(df))

Fraction of missing values in the PUlocationID coumn: 0.7710194817407081


In [ ]:


categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)